In [1]:
import re
import os
import pickle
from collections import defaultdict

In [2]:
gas_phase_logs = os.popen("find /home/michael/gas_molsimps/ -type f -name '*.log'").readlines()
gas_phase_logs = [x[:-1] for x in gas_phase_logs]

valid_cats = [x.split("/")[4] for x in gas_phase_logs if "record.log" not in x]
valid_cats = list(set(valid_cats))

In [3]:
def get_e(logfile):
    with open(logfile, "r") as f:
        lines = f.readlines()
        
    for line in lines:
        if "SCF Done" in line:
            energy = float(re.findall(r"[-+]?\d*\.\d+|\d+", line)[0])
    return energy

gas_phase_energy = defaultdict(float)
for log in gas_phase_logs:
    if "record.log" in log:
        continue
    #print(log)
    gas_phase_energy[log] = get_e(log)
    if "Ru_oct_desorbed_1" in log:
        print(get_e(log))

-1801.30106517
-1877.09214823
-1952.23834052


In [4]:
from collections import defaultdict

smd_energies = pickle.load(open("/home/michael/PycharmProjects/gaussian_job_manage/ele_ene4solv_correction.p", "rb"))

solvent_corr = defaultdict(float)
for key in smd_energies:
    key4gas = key.replace(".log", "gas.log")
    key4gas = key4gas.replace("ht_scaling_results", "gas_molsimps")
    if gas_phase_energy[key4gas]==0 or "Cr_32a_35" in key:
        continue
    else:
        key2add = ''.join(key.replace("ht_scaling_results", "beef_molsimpsbeef_molsimps").split("/")[:-1])
        cat_details = key2add[37:]
        if "VAC" in cat_details:
            cat_key = cat_details[:cat_details.index("V")] + "VAC"
        if "OH" in cat_details:
            cat_key = cat_details[:cat_details.index("OH")] + "OH"
        if "OOH" in cat_details:
            cat_key = cat_details[:cat_details.index("OOH")] + "OOH"
        #print(cat_key)
        solvent_corr[cat_key] = smd_energies[key] - gas_phase_energy[key4gas]
        #break
    #print(smd_energies[key] - gas_phase_energy[key4gas])


In [5]:
completed_cats = list(set([x.split("/")[4] for x in gas_phase_logs if "record.log" not in x]))

In [6]:
solvent_corr

defaultdict(float,
            {'Fe_31a_17VAC': -0.05670185000008132,
             'Fe_31a_17OH': -0.05793466999989505,
             'Fe_31a_17OOH': -0.05688666000014564,
             'Co_31a_23VAC': -0.09960406000027433,
             'Co_31a_23OH': -0.09912147000000004,
             'Co_31a_23OOH': -0.09721791999982088,
             'Mn_32a_35VAC': -0.11194931999989421,
             'Mn_32a_35OH': -0.08402855999997882,
             'Mn_32a_35OOH': -0.09630766999998741,
             'Ru_32a_34VAC': -0.12774591999982476,
             'Ru_32a_34OH': -0.11332332000006318,
             'Ru_32a_34OOH': -0.11263790999987577,
             'Mn_31a_23VAC': -0.11548162000008233,
             'Mn_31a_23OH': -0.10944921999998769,
             'Mn_31a_23OOH': -0.10464877999993405,
             'Fe_3a1_5_VAC': -0.22415154999998776,
             'Fe_3a1_5_OH': -0.20930468999995355,
             'Fe_3a1_5_OOH': -0.206233549999979,
             'Ni_3a1_5_VAC': -0.20863355000005868,
             'Ni_3a1

In [7]:
pickle.dump(solvent_corr, open("solvent_correction.p", "wb"), protocol=2)